# Import dataset and lib


A dataset of 100,000 Arabic customer reviews labeled for sentiment analysis (positive/negative). It is ideal for training and testing NLP models.

In [ ]:
!kaggle datasets download abedkhooli/arabic-100k-reviews

Dataset URL: https://www.kaggle.com/datasets/abedkhooli/arabic-100k-reviews
License(s): copyright-authors
 29% 5.00M/17.0M [00:00<00:00, 43.7MB/s]
100% 17.0M/17.0M [00:00<00:00, 81.3MB/s]


In [ ]:
!unzip arabic-100k-reviews.zip

Archive:  arabic-100k-reviews.zip
  inflating: ar_reviews_100k.tsv     


In [ ]:
!pip install pyarabic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 4.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import nltk
import pyarabic # This lib for dealing with arabic words for NLP tasks
from pyarabic.araby import strip_tashkeel , strip_tatweel
from pyarabic.araby import tokenize
import re
from wordcloud import WordCloud
import tensorflow as tf

# EDA and pre-processing

In [ ]:
df = pd.read_csv('/content/ar_reviews_100k.tsv', sep='\t')

In [ ]:
df.head()

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   99999 non-null  object
 1   text    99999 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


In [ ]:
df.isnull().sum()

,0
label,0
text,0


In [ ]:
df['label'].value_counts()

,count
label,
Positive,33333
Mixed,33333
Negative,33333


### Pre processing

In [ ]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('arabic')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stopwords

['إذ',
 'إذا',
 'إذما',
 'إذن',
 'أف',
 'أقل',
 'أكثر',
 'ألا',
 'إلا',
 'التي',
 'الذي',
 'الذين',
 'اللاتي',
 'اللائي',
 'اللتان',
 'اللتيا',
 'اللتين',
 'اللذان',
 'اللذين',
 'اللواتي',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'أم',
 'أما',
 'أما',
 'إما',
 'أن',
 'إن',
 'إنا',
 'أنا',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'إنما',
 'إنه',
 'أنى',
 'أنى',
 'آه',
 'آها',
 'أو',
 'أولاء',
 'أولئك',
 'أوه',
 'آي',
 'أي',
 'أيها',
 'إي',
 'أين',
 'أين',
 'أينما',
 'إيه',
 'بخ',
 'بس',
 'بعد',
 'بعض',
 'بك',
 'بكم',
 'بكم',
 'بكما',
 'بكن',
 'بل',
 'بلى',
 'بما',
 'بماذا',
 'بمن',
 'بنا',
 'به',
 'بها',
 'بهم',
 'بهما',
 'بهن',
 'بي',
 'بين',
 'بيد',
 'تلك',
 'تلكم',
 'تلكما',
 'ته',
 'تي',
 'تين',
 'تينك',
 'ثم',
 'ثمة',
 'حاشا',
 'حبذا',
 'حتى',
 'حيث',
 'حيثما',
 'حين',
 'خلا',
 'دون',
 'ذا',
 'ذات',
 'ذاك',
 'ذان',
 'ذانك',
 'ذلك',
 'ذلكم',
 'ذلكما',
 'ذلكن',
 'ذه',
 'ذو',
 'ذوا',
 'ذواتا',
 'ذواتي',
 'ذي',
 'ذين',
 'ذينك',
 'ريث',
 'سوف',
 'سوى',
 'شتان',
 'عدا',
 'عسى',
 'عل'

In [ ]:
df.sample(10)

,label,text
79376,Negative,مخيب للأمل. أن الفندق جديد والمرافق العامة الف...
499,Positive,ممتاز. اولا تعامل موضفوا الفندق مع النزلاء وال...
62613,Mixed,عجبنى انى عرفت ابطال حقيقين اول مرة كنت اقرأ ا...
68203,Negative,سواليف . كان المفروض تكون هذه قراءتي الأولى وأ...
94174,Negative,مكتوب على الغلاف رواية، لكنها ابتعدت كل البعد ...
72662,Negative,الكتاب ممل بالرغم من حبي الشديد لكتابات د. أحم...
1870,Positive,فيلم متكامل للعائلة فيلم شامل رمز للنجاح بكل م...
11678,Positive,يارب بعودة العمرة . المطعم بحاجة الى تجويد الا...
88265,Negative,من اسوأ السير الذاتيه التي قرأتها بحياتي
12168,Positive,استثنائي. الكل متعاون و ودود وخصوصا موظف الاست...


In [ ]:
spaces = r'\s+'
dots = r'[^\w\s]'

no_need = r'|'.join([spaces,dots])
df['text'] = df['text'].apply(lambda x: re.sub(no_need," ",x))

In [ ]:
def preprocess(text):
  text = strip_tatweel(text)  # Remove ـــ from the words such as (انــــس) --> (انس)
  text = strip_tashkeel(text)  # Remove َ ً ُ ٌ  such as (أنٌسَ) --> (انس)
  text = pyarabic.araby.normalize_hamza(text)  # ء --> ئ
  text = tokenize(text)  # Tokenize the text into a list of words
  text = [word for word in text if word not in stopwords]  # Remove stopwords
  return text

In [ ]:
df['text'] = df['text'].apply(preprocess)

In [ ]:
df.head()

,label,text
0,Positive,"[ممتاز, نوعا, النظافة, والموقع, والتجهيز, والش..."
1,Positive,"[ءحد, ءسباب, نجاح, الءمارات, ءن, شخص, الدولة, ..."
2,Positive,"[هادفة, وقوية, تنقلك, صخب, شوارع, القاهرة, الى..."
3,Positive,"[خلصنا, مبدءيا, اللي, مستني, ابهار, زي, الفيل,..."
4,Positive,"[ياسات, جلوريا, جزء, يتجزء, دبي, فندق, متكامل,..."


In [ ]:
from collections import Counter

# Flatten the list of lists in 'text' column into a single list of words
all_words = [word for sublist in df['text'].tolist() for word in sublist]

# Now use the Counter on the flattened list
word_counts = Counter(all_words)

In [ ]:
word_counts.most_common(10)

[('ءن', 43008),
 ('جدا', 34057),
 ('ان', 26321),
 ('الكتاب', 25098),
 ('الرواية', 24653),
 ('فى', 22425),
 ('الفندق', 17560),
 ('كانت', 14361),
 ('ءلى', 14181),
 ('الكاتب', 12359)]

In [ ]:
from sklearn.preprocessing import LabelEncoder

df['label'] = LabelEncoder().fit_transform(df['label'])


In [ ]:
df.head()

,label,text
0,2,"[ممتاز, نوعا, النظافة, والموقع, والتجهيز, والش..."
1,2,"[ءحد, ءسباب, نجاح, الءمارات, ءن, شخص, الدولة, ..."
2,2,"[هادفة, وقوية, تنقلك, صخب, شوارع, القاهرة, الى..."
3,2,"[خلصنا, مبدءيا, اللي, مستني, ابهار, زي, الفيل,..."
4,2,"[ياسات, جلوريا, جزء, يتجزء, دبي, فندق, متكامل,..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

x = df['text']
y = df['label']

# Convert the lists of words in 'text' column back to strings
x = x.apply(lambda tokens: ' '.join(tokens))

vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(x)

In [ ]:
x

<99999x305290 sparse matrix of type '<class 'numpy.float64'>'
	with 3700451 stored elements in Compressed Sparse Row format>

# Building the models

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression

# Initialize and train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))

Accuracy: 0.66745
              precision    recall  f1-score   support

           0       0.59      0.56      0.58      6722
           1       0.72      0.73      0.72      6603
           2       0.69      0.72      0.70      6675

    accuracy                           0.67     20000
   macro avg       0.67      0.67      0.67     20000
weighted avg       0.67      0.67      0.67     20000



In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Initialize and train the model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Make predictions
d_y_pred = model.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, d_y_pred))

print(classification_report(y_test, d_y_pred))

Accuracy: 0.56315
              precision    recall  f1-score   support

           0       0.52      0.50      0.51      6722
           1       0.60      0.58      0.59      6603
           2       0.58      0.60      0.59      6675

    accuracy                           0.56     20000
   macro avg       0.56      0.56      0.56     20000
weighted avg       0.56      0.56      0.56     20000

